In [1]:
import os
import sys
import json
import glob
import math
import torch
import torch.nn as nn
import torch.nn.functional as F
from reformer_pytorch import Reformer

In [2]:
base_dir = os.path.dirname(os.getcwd())
data_dir = os.path.join(base_dir, "data", "original")
train_files = glob.glob(os.path.join(data_dir, "train", "*", "*.obj"))
valid_files = glob.glob(os.path.join(data_dir, "val", "*", "*.obj"))
print(len(train_files), len(valid_files))

src_dir = os.path.join(base_dir, "src")
sys.path.append(os.path.join(src_dir))

7003 1088


In [3]:
from utils_polygen import load_pipeline
from tokenizers import EncodeVertexTokenizer, FaceTokenizer

In [4]:
v_batch, f_batch = [], []
for i in range(3):
    vs, _, fs = load_pipeline(train_files[i])
    
    vs = torch.tensor(vs)
    fs = [torch.tensor(f) for f in fs]
    
    v_batch.append(vs)
    f_batch.append(fs)
    print(vs.shape, len(fs))
    print("="*60)

torch.Size([204, 3]) 160
torch.Size([62, 3]) 45
torch.Size([64, 3]) 601


In [5]:
enc_tokenizer = EncodeVertexTokenizer(max_seq_len=2592)
dec_tokenizer = FaceTokenizer(max_seq_len=3936)

In [6]:
src_tokens = enc_tokenizer.tokenize(v_batch)
src_tokens

{'value_tokens': tensor([[167, 122, 167,  ...,   0,   0,   0],
         [165, 164, 165,  ...,   0,   0,   0],
         [165, 165, 128,  ...,   0,   0,   0]]),
 'coord_type_tokens': tensor([[1, 2, 3,  ..., 0, 0, 0],
         [1, 2, 3,  ..., 0, 0, 0],
         [1, 2, 3,  ..., 0, 0, 0]]),
 'position_tokens': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'padding_mask': tensor([[False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True],
         [False, False, False,  ...,  True,  True,  True]])}

In [7]:
for k, v in src_tokens.items():
    print(k, v.shape)

value_tokens torch.Size([3, 2592])
coord_type_tokens torch.Size([3, 2592])
position_tokens torch.Size([3, 2592])
padding_mask torch.Size([3, 2592])


In [8]:
class FaceEncoderEmbedding(nn.Module):
    
    def __init__(self, embed_dim=256,
                 vocab_value=259, pad_idx_value=2, 
                 vocab_coord_type=4, pad_idx_coord_type=0,
                 vocab_position=1000, pad_idx_position=0):
        
        super().__init__()
        
        self.value_embed = nn.Embedding(
            vocab_value, embed_dim, padding_idx=pad_idx_value
        )
        self.coord_type_embed = nn.Embedding(
            vocab_coord_type, embed_dim, padding_idx=pad_idx_coord_type
        )
        self.position_embed = nn.Embedding(
            vocab_position, embed_dim, padding_idx=pad_idx_position
        )
        
        self.embed_scaler = math.sqrt(embed_dim)
        
    def forward(self, tokens):
        
        """get embedding for Face Encoder.
        
        Args
            tokens [dict]: tokenized vertex info.
                `value_tokens` [torch.tensor]:
                        padded (batch, length) shape long tensor
                        with coord value from 0 to 2^n(bit).
                `coord_type_tokens` [torch.tensor]:
                        padded (batch, length) shape long tensor implies x or y or z.
                `position_tokens` [torch.tensor]:
                        padded (batch, length) shape long tensor
                        representing coord position (NOT sequence position).
        
        Returns
            embed [torch.tensor]: (batch, length, embed) shape tensor after embedding.
                        
        """
              
        embed = self.value_embed(tokens["value_tokens"])
        embed = embed + self.coord_type_embed(tokens["coord_type_tokens"])
        embed = embed + self.position_embed(tokens["position_tokens"])
        embed = embed * self.embed_scaler
        
        embed = embed[:, :-1]
        embed = torch.cat([
            e.sum(dim=1).unsqueeze(dim=1) for e in embed.split(3, dim=1)
        ], dim=1)
        
        return embed
    
    def forward_original(self, tokens):
        # original PolyGen embedding did something like this (no position info?).
        embed = self.value_embed(tokens["value_tokens"]) * self.embed_scaler
        embed = torch.cat([
            e.sum(dim=1).unsqueeze(dim=1) for e in embed[:, :-1].split(3, dim=1)
        ], dim=1)
        return embed

In [9]:
src_embed = FaceEncoderEmbedding(embed_dim=128)

In [10]:
print(
    src_tokens["value_tokens"].shape,
    src_tokens["coord_type_tokens"].shape,
    src_tokens["position_tokens"].shape,
    src_tokens["padding_mask"].shape
)

torch.Size([3, 2592]) torch.Size([3, 2592]) torch.Size([3, 2592]) torch.Size([3, 2592])


In [11]:
src_emb = src_embed.forward_original(src_tokens)
src_emb.shape

torch.Size([3, 864, 128])

In [12]:
src_emb = src_embed(src_tokens)
src_emb.shape

torch.Size([3, 864, 128])

In [13]:
tgt_tokens = dec_tokenizer.tokenize(f_batch)
tgt_tokens

{'value_tokens': tensor([[  0, 206, 205,  ...,   2,   2,   2],
         [  0,  64,  63,  ...,   2,   2,   2],
         [  0,  66,  66,  ...,   2,   2,   2]]),
 'target_tokens': tensor([[206, 205, 203,  ...,   2,   2,   2],
         [ 64,  63,  61,  ...,   2,   2,   2],
         [ 66,  66,  64,  ...,   2,   2,   2]]),
 'in_position_tokens': tensor([[1, 2, 3,  ..., 0, 0, 0],
         [1, 2, 3,  ..., 0, 0, 0],
         [1, 2, 3,  ..., 0, 0, 0]]),
 'out_position_tokens': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 'ref_v_mask': tensor([[0., 1., 1.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.],
         [0., 1., 1.,  ..., 0., 0., 0.]]),
 'ref_v_ids': tensor([[  0, 203, 202,  ...,   0,   0,   0],
         [  0,  61,  60,  ...,   0,   0,   0],
         [  0,  63,  63,  ...,   0,   0,   0]]),
 'ref_e_mask': tensor([[1., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 0.,  ..., 1., 1., 1.],
         [1., 0., 0.,  ..., 1.,

In [14]:
class FaceDecoderEmbedding(nn.Module):
    
    def __init__(self, embed_dim=256,
                 vocab_value=3, pad_idx_value=2, 
                 vocab_in_position=100, pad_idx_in_position=0,
                 vocab_out_position=1000, pad_idx_out_position=0):
        
        super().__init__()
        
        self.value_embed = nn.Embedding(
            vocab_value, embed_dim, padding_idx=pad_idx_value
        )
        self.in_position_embed = nn.Embedding(
            vocab_in_position, embed_dim, padding_idx=pad_idx_in_position
        )
        self.out_position_embed = nn.Embedding(
            vocab_out_position, embed_dim, padding_idx=pad_idx_out_position
        )
        
        self.embed_scaler = math.sqrt(embed_dim)
        
    def forward(self, encoder_embed, tokens):
        
        """get embedding for Face Decoder.
        note that value_embeddings consist of two embedding.
          - pointer to encoder outputs
          - embedding for special tokens such as <end-of-face>, <eos>, <pad>.
        
        Args
            encoder_embed [torch.tensor]:
                    (batch, src-length, embed) shape tensor from encoder.
            tokens [dict]: all contents are in the shape of (batch, tgt-length).
                `ref_v_ids` [torch.tensor]:
                        this is used as pointer to `encoder_embed`.
                `ref_v_mask` [torch.tensor]:
                        mask for special token positions in pointer embeddings. 
                `ref_e_ids` [torch.tensor]:
                        embed ids for special tokens.
                `ref_e_ids` [torch.tensor]:
                        mask for pointer token position in special token embeddings.
                `in_position_tokens` [torch.tensor]:
                        embed ids for positions in face.
                `out_position_tokens` [torch.tensor]:
                        embed ids for positions of face itself in sequence.
                        
        Returns
            embed [torch.tensor]: (batch, tgt-length, embed) shape tensor of embeddings.
                        
        """
        
        embed = torch.cat([
            encoder_embed[b_idx, ids].unsqueeze(dim=0) 
            for b_idx, ids in enumerate(tokens["ref_v_ids"].unbind(dim=0))
        ], dim=0)
        embed = embed * tokens["ref_v_mask"].unsqueeze(dim=2)
        
        additional_embeddings = self.value_embed(tokens["ref_e_ids"]) * tokens["ref_e_mask"].unsqueeze(dim=2)
        additional_embeddings = additional_embeddings + self.in_position_embed(tokens["in_position_tokens"])
        additional_embeddings = additional_embeddings + self.out_position_embed(tokens["out_position_tokens"])
        additional_embeddings = additional_embeddings * self.embed_scaler
        
        embed = embed + additional_embeddings
        return embed

In [15]:
tgt_embed = FaceDecoderEmbedding(embed_dim=128)

In [16]:
tgt_emb = tgt_embed(src_emb, tgt_tokens)
tgt_emb.shape

torch.Size([3, 3936, 128])

In [17]:
torch.bmm(tgt_emb, src_emb.permute(0, 2, 1)).shape

torch.Size([3, 3936, 864])

In [18]:
tgt_embed.value_embed.weight[None, ...].repeat(3, 1, 1).shape

torch.Size([3, 3, 128])

In [19]:
BATCH = 3
test = torch.cat([
    tgt_embed.value_embed.weight[None, ...].repeat(BATCH, 1, 1),
    src_emb
], dim=1)
print(test.shape)
print(test[:, 2]) # id 2 is padding.

torch.Size([3, 867, 128])
tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [20]:
torch.bmm(tgt_emb, test.permute(0, 2, 1)).shape

torch.Size([3, 3936, 867])

In [21]:
class Config(object):
    
    def write_to_json(self, out_path):
        with open(out_path, "w") as fw:
            json.dump(self.config, fw, indent=4)
            
    def load_from_json(self, file_path):
        with open(file_path) as fr:
            self.config = json.load(fr)
        
    def __getitem__(self, key):
        return self.config[key]

In [22]:
class FacePolyGenConfig(Config):
    
    def __init__(self,
                 embed_dim=256, 
                 src__max_seq_len=2400, 
                 src__tokenizer__pad_id=0,
                 tgt__max_seq_len=3900,
                 tgt__tokenizer__bof_id=0,
                 tgt__tokenizer__eos_id=1, 
                 tgt__tokenizer__pad_id=2,
                 src__embedding__vocab_value=256 + 3, 
                 src__embedding__vocab_coord_type=4, 
                 src__embedding__vocab_position=1000, 
                 src__embedding__pad_idx_value=2,
                 src__embedding__pad_idx_coord_type=0,
                 src__embedding__pad_idx_position=0,
                 tgt__embedding__vocab_value=3,
                 tgt__embedding__vocab_in_position=100,
                 tgt__embedding__vocab_out_position=1000,
                 tgt__embedding__pad_idx_value=2,
                 tgt__embedding__pad_idx_in_position=0,
                 tgt__embedding__pad_idx_out_position=0,
                 src__reformer__depth=12,
                 src__reformer__heads=8,
                 src__reformer__n_hashes=8,
                 src__reformer__bucket_size=48,
                 src__reformer__causal=True,
                 src__reformer__lsh_dropout=0.2, 
                 src__reformer__ff_dropout=0.2,
                 src__reformer__post_attn_dropout=0.2,
                 src__reformer__ff_mult=4,
                 tgt__reformer__depth=12,
                 tgt__reformer__heads=8,
                 tgt__reformer__n_hashes=8,
                 tgt__reformer__bucket_size=48,
                 tgt__reformer__causal=True,
                 tgt__reformer__lsh_dropout=0.2, 
                 tgt__reformer__ff_dropout=0.2,
                 tgt__reformer__post_attn_dropout=0.2,
                 tgt__reformer__ff_mult=4):
        
        # auto padding for max_seq_len
        src_denominator = (src__reformer__bucket_size * 2 * 3)
        if src__max_seq_len % src_denominator != 0:
            divisables = src__max_seq_len // src_denominator + 1
            src__max_seq_len_new = divisables * src_denominator
            print("src__max_seq_len changed, because of lsh-attention's bucket_size")
            print("before: {} --> after: {} (with bucket_size: {})".format(
                src__max_seq_len, src__max_seq_len_new, src__reformer__bucket_size
            ))
            src__max_seq_len = src__max_seq_len_new
            
        tgt_denominator = tgt__reformer__bucket_size * 2
        if tgt__max_seq_len % tgt_denominator != 0:
            divisables = tgt__max_seq_len // tgt_denominator + 1
            tgt__max_seq_len_new = divisables * tgt_denominator
            print("tgt__max_seq_len changed, because of lsh-attention's bucket_size")
            print("before: {} --> after: {} (with bucket_size: {})".format(
                tgt__max_seq_len, tgt__max_seq_len_new, tgt__reformer__bucket_size
            ))
            tgt__max_seq_len = tgt__max_seq_len_new
        
        
        # tokenizer config
        src_tokenizer_config = {
            "pad_id": src__tokenizer__pad_id,
            "max_seq_len": src__max_seq_len,
        }
        tgt_tokenizer_config = {
            "bof_id": tgt__tokenizer__bof_id,
            "eos_id": tgt__tokenizer__eos_id,
            "pad_id": tgt__tokenizer__pad_id,
            "max_seq_len": tgt__max_seq_len,
        }
        
        # embedding config
        src_embedding_config = {
            "vocab_value": src__embedding__vocab_value,
            "vocab_coord_type": src__embedding__vocab_coord_type,
            "vocab_position": src__embedding__vocab_position,
            "pad_idx_value": src__embedding__pad_idx_value,
            "pad_idx_coord_type": src__embedding__pad_idx_coord_type,
            "pad_idx_position": src__embedding__pad_idx_position,
            "embed_dim": embed_dim,
        }
        tgt_embedding_config = {
            "vocab_value": tgt__embedding__vocab_value,
            "vocab_in_position": tgt__embedding__vocab_in_position,
            "vocab_out_position": tgt__embedding__vocab_out_position,
            "pad_idx_value": tgt__embedding__pad_idx_value,
            "pad_idx_in_position": tgt__embedding__pad_idx_in_position,
            "pad_idx_out_position": tgt__embedding__pad_idx_out_position,
            "embed_dim": embed_dim,
        }
        
        # reformer info
        src_reformer_config = {
            "dim": embed_dim,
            "max_seq_len": src__max_seq_len,
            "depth": src__reformer__depth,
            "heads": src__reformer__heads,
            "bucket_size": src__reformer__bucket_size,
            "n_hashes": src__reformer__n_hashes,
            "causal": src__reformer__causal,
            "lsh_dropout": src__reformer__lsh_dropout, 
            "ff_dropout": src__reformer__ff_dropout,
            "post_attn_dropout": src__reformer__post_attn_dropout,
            "ff_mult": src__reformer__ff_mult,
        }
        
        tgt_reformer_config = {
            "dim": embed_dim,
            "max_seq_len": tgt__max_seq_len,
            "depth": tgt__reformer__depth,
            "heads": tgt__reformer__heads,
            "bucket_size": tgt__reformer__bucket_size,
            "n_hashes": tgt__reformer__n_hashes,
            "causal": tgt__reformer__causal,
            "lsh_dropout": tgt__reformer__lsh_dropout, 
            "ff_dropout": tgt__reformer__ff_dropout,
            "post_attn_dropout": tgt__reformer__post_attn_dropout,
            "ff_mult": tgt__reformer__ff_mult,
        }
        
        self.config = {
            "embed_dim": embed_dim,
            "src_tokenizer": src_tokenizer_config,
            "tgt_tokenizer": tgt_tokenizer_config,
            "src_embedding": src_embedding_config,
            "tgt_embedding": tgt_embedding_config,
            "src_reformer": src_reformer_config,
            "tgt_reformer": tgt_reformer_config,
        }

In [23]:
# utility functions

def accuracy(y_pred, y_true, ignore_label=None, device=None):
    y_pred = y_pred.argmax(dim=1)

    if ignore_label:
        normalizer = torch.sum(y_true!=ignore_label)
        ignore_mask = torch.where(
            y_true == ignore_label,
            torch.zeros_like(y_true, device=device),
            torch.ones_like(y_true, device=device)
        ).type(torch.float32)
    else:
        normalizer = y_true.shape[0]
        ignore_mask = torch.ones_like(y_true, device=device).type(torch.float32)

    acc = (y_pred.reshape(-1)==y_true.reshape(-1)).type(torch.float32)
    acc = torch.sum(acc*ignore_mask)
    return acc / normalizer


def init_weights(m):
    if type(m) == nn.Linear:
        nn.init.xavier_normal_(m.weight)
    if type(m) == nn.Embedding:
        nn.init.uniform_(m.weight, -0.05, 0.05)

In [31]:
class FacePolyGen(nn.Module):
    
    def __init__(self, model_config):
        super().__init__()
        self.src_tokenizer = EncodeVertexTokenizer(**model_config["src_tokenizer"])
        self.tgt_tokenizer = FaceTokenizer(**model_config["tgt_tokenizer"])
        
        self.src_embedding = FaceEncoderEmbedding(**model_config["src_embedding"])
        self.tgt_embedding = FaceDecoderEmbedding(**model_config["tgt_embedding"])
        
        self.src_reformer = Reformer(**model_config["src_reformer"])
        self.tgt_reformer = Reformer(**model_config["tgt_reformer"])
        
        self.src_norm = nn.LayerNorm(model_config["embed_dim"])
        self.tgt_norm = nn.LayerNorm(model_config["embed_dim"])
        self.loss_func = nn.CrossEntropyLoss(ignore_index=model_config["tgt_tokenizer"]["pad_id"])
        
        self.apply(init_weights)
        self.embed_scaler = math.sqrt(model_config["embed_dim"])
    
    def encode(self, src_tokens, device=None):
        
        """forward function which can be used for both train/predict.
        this function only encodes vertex information
        because decoders behave as really auto-regressive function.
        
        Args
            src_tokens [dict]: tokenized vertex info.
                `value_tokens` [torch.tensor]:
                        padded (batch, src-length) shape long tensor
                        with coord value from 0 to 2^n(bit).
                `coord_type_tokens` [torch.tensor]:
                        padded (batch, src-length) shape long tensor implies x or y or z.
                `position_tokens` [torch.tensor]:
                        padded (batch, src-length) shape long tensor
                        representing coord position (NOT sequence position).
                `padding_mask` [torch.tensor]:
                        (batch, src-length) shape mask implies <pad> tokens.
        
        Returns
            hs [torch.tensor]: (batch, src-length, embed) shape tensor after encoder.
        
        """
        
        hs = self.src_embedding(src_tokens)
        hs = self.src_reformer(
            hs, input_mask=src_tokens["padding_mask"]
        )
        hs = self.src_norm(hs)
        
        # calc pointing to vertex
        BATCH = hs.shape[0]
        sptk_embed = self.tgt_embedding.value_embed.weight
        encoder_embed_with_sptk = torch.cat([
            sptk_embed[None, ...].repeat(BATCH, 1, 1), hs
        ], dim=1)
        
        return hs, encoder_embed_with_sptk
    
    def decode(self, encoder_embed, encoder_embed_with_sptk, tgt_tokens, pred_idx=None, device=None):
        hs = self.tgt_embedding(encoder_embed, tgt_tokens)
        hs = self.tgt_reformer(
            hs, input_mask=tgt_tokens["padding_mask"]
        )
        hs = self.tgt_norm(hs)        
        
        if pred_idx is None:
            hs = torch.bmm(
                hs, encoder_embed_with_sptk.permute(0, 2, 1))
        else:
            hs = torch.bmm(
                hs[:, pred_idx:pred_idx+1],
                encoder_embed_with_sptk.permute(0, 2, 1)
            )
        return hs
        
        
    def forward(self, inputs, device=None):
        
        """Calculate loss while training.
        
        Args
            inputs [dict]: dict containing batched inputs.
                `vertices` [list(torch.tensor)]:
                        variable-length-list of 
                        (length, 3) shaped tensor of quantized-vertices.
                `faces` [list(list(torch.tensor))]:
                        batch-length-list of
                        variable-length-list (per face) of 
                        (length,) shaped vertex-ids which constructs a face.
            device [torch.device]: gpu or not gpu, that's the problem.
                
        Returns
            outputs [dict]: dict containing calculated variables.
                `loss` [torch.tensor]:
                        calculated scalar-shape loss with backprop info.
                `accuracy` [torch.tensor]:
                        calculated scalar-shape accuracy.
            
        """
        
        src_tokens = self.src_tokenizer.tokenize(inputs["vertices"])
        src_tokens = {k: v.to(device) for k, v in src_tokens.items()}
        
        tgt_tokens = self.tgt_tokenizer.tokenize(inputs["faces"])
        tgt_tokens = {k: v.to(device) for k, v in tgt_tokens.items()}
        
        encoder_embed, encoder_embed_with_sptk = self.encode(src_tokens, device=device)
        decoder_embed = self.decode(encoder_embed, encoder_embed_with_sptk, tgt_tokens, device=device)
        
        BATCH, TGT_LENGTH, SRC_LENGTH = decoder_embed.shape
        decoder_embed = decoder_embed.reshape(BATCH*TGT_LENGTH, SRC_LENGTH)
        targets = tgt_tokens["target_tokens"].reshape(BATCH*TGT_LENGTH,)
        
        acc = accuracy(
            decoder_embed, targets, ignore_label=self.tgt_tokenizer.pad_id, device=device
        )
        loss = self.loss_func(decoder_embed, targets)
        
        if hasattr(self, 'reporter'):
            self.reporter.report({
                "accuracy": acc.item(),
                "perplexity": torch.exp(loss).item(),
                "loss": loss.item(),
            })
        
        output = {
            "accuracy": acc.item(),
            "perplexity": torch.exp(loss).item(),
            "loss": loss.item(),
        }

        return loss, output
    
    @torch.no_grad()
    def predict(self, inputs, max_seq_len=3936, top_p=0.9, seed=0, device=None):
        
        # setting for sampling reproducibility.
        if torch.cuda.is_available():
            torch.cuda.manual_seed(seed)
        torch.manual_seed(seed)
        torch.set_deterministic(True)
        
        
        tgt_tokenizer = self.tgt_tokenizer
        special_tokens = tgt_tokenizer.special_tokens
        
        # calc vertex encoding first.
        src_tokens = self.src_tokenizer.tokenize(inputs["vertices"])
        src_tokens = {k: v.to(device) for k, v in src_tokens.items()}
        
        encoder_embed, encoder_embed_with_sptk = self.encode(src_tokens, device=device)
        
        # prepare for generation.
        tgt_tokens = model.tgt_tokenizer.tokenize([[torch.tensor([], dtype=torch.int32)]])
        tgt_tokens["value_tokens"][:, 1] = model.tgt_tokenizer.special_tokens["pad"]
        tgt_tokens["ref_e_ids"][:, 1] = model.tgt_tokenizer.special_tokens["pad"]
        tgt_tokens["padding_mask"][:, 1] = True
        
        output_vocab_length = encoder_embed_with_sptk.shape[1]
        preds = [torch.tensor([], dtype=torch.int32)]
        history_in_face = torch.zeros((1, output_vocab_length), dtype=torch.bool)
        pred_idx = 0
        now_face_idx = 0
        
        try:
            while (pred_idx <= max_seq_len-1):
                print(pred_idx, end=", ")

                if pred_idx >= 1:
                    tgt_tokens = tgt_tokenizer.tokenize([[torch.cat([p]) for p in preds]])
                    tgt_tokens["value_tokens"][:, pred_idx+1] = special_tokens["pad"]
                    tgt_tokens["ref_e_ids"][:, pred_idx+1] = special_tokens["pad"]
                    tgt_tokens["padding_mask"][:, pred_idx+1] = True

                hs = self.decode(encoder_embed, encoder_embed_with_sptk, tgt_tokens, pred_idx=pred_idx, device=device)
                hs = hs[:, 0]

                ##### greedy sampling
                # pred = hs.argmax(dim=1)

                ### top-p sampling
                hs = torch.where(
                    history_in_face,
                    torch.full_like(hs, -np.inf, device=device),
                    hs
                )
                probas, indeces = torch.sort(hs, dim=1, descending=True)
                cum_probas = torch.cumsum(F.softmax(probas, dim=1), dim=1)

                condition = cum_probas <= top_p
                if condition.sum() == 0:
                    candidates = torch.full_like(probas, -np.inf, device=device)
                    candidates[:, 0] = 1.
                else:
                    candidates = torch.where(
                        condition, probas, torch.full_like(probas, -np.inf, device=device)
                    )

                probas = F.softmax(candidates, dim=1)
                pred = indeces[0, torch.multinomial(probas, 1).squeeze(dim=1)]

                if pred == special_tokens["eos"]:
                    break
                if pred == special_tokens["bof"]:
                    now_face_idx += 1
                    history_in_face = torch.arange(output_vocab_length) > preds[-1][0]+len(special_tokens)
                    history_in_face = history_in_face[None, :]
                    preds.append(torch.tensor([], dtype=torch.int32))
                else:
                    history_in_face[:, pred] = True
                    preds[now_face_idx] = \
                        torch.cat([preds[now_face_idx], pred-len(special_tokens)])
                pred_idx += 1
        
        except KeyboardInterrupt:
            return preds
        
        return preds

In [32]:
inputs = {
    "vertices": v_batch[:2],
    "faces": f_batch[:2],
}
for k, values in inputs.items():
    print(k)
    for v in values:
        if k == "vertices":
            print(v.shape)
        else:
            print(len(v))

vertices
torch.Size([204, 3])
torch.Size([62, 3])
faces
160
45


In [33]:
config = FacePolyGenConfig(
    embed_dim=64, 
    src__reformer__depth=4, src__reformer__lsh_dropout=0.,
    src__reformer__ff_dropout=0., src__reformer__post_attn_dropout=0.,
    tgt__reformer__depth=4, tgt__reformer__lsh_dropout=0.,
    tgt__reformer__ff_dropout=0., tgt__reformer__post_attn_dropout=0.
)
model = FacePolyGen(config)

src__max_seq_len changed, because of lsh-attention's bucket_size
before: 2400 --> after: 2592 (with bucket_size: 48)
tgt__max_seq_len changed, because of lsh-attention's bucket_size
before: 3900 --> after: 3936 (with bucket_size: 48)


In [34]:
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)

In [28]:
import numpy as np
epoch_num = 300
model.train()
losses = []
accs = []
perps = []

for i in range(epoch_num):
    optimizer.zero_grad()
    loss, outputs = model(inputs)
    
    ls = outputs["loss"]
    acc = outputs["accuracy"]
    perp = outputs["perplexity"]
    losses.append(ls)
    accs.append(acc)
    perps.append(perp)
    
    if i % 10 == 0:
        ave_loss = np.mean(losses[-10:])
        ave_acc = np.mean(accs[-10:])
        ave_perp = np.mean(perps[-10:])
        print("iteration: {}\tloss: {:.5f}\tperp: {:.3f}\tacc: {:.5f}".format(
            i, ave_loss, ave_perp, ave_acc))
    
    loss.backward()
    optimizer.step()

iteration: 0	loss: 26.36524	perp: 282018414592.000	acc: 0.02822
iteration: 10	loss: 15.68826	perp: 1149463465.527	acc: 0.02730
iteration: 20	loss: 8.15299	perp: 9884.403	acc: 0.03627
iteration: 30	loss: 4.42965	perp: 90.194	acc: 0.15328
iteration: 40	loss: 3.80131	perp: 46.192	acc: 0.19427
iteration: 50	loss: 3.29365	perp: 27.172	acc: 0.27295
iteration: 60	loss: 2.89013	perp: 18.110	acc: 0.30979
iteration: 70	loss: 2.54781	perp: 12.829	acc: 0.33743
iteration: 80	loss: 2.27219	perp: 9.726	acc: 0.36158
iteration: 90	loss: 2.04413	perp: 7.737	acc: 0.37651
iteration: 100	loss: 1.84244	perp: 6.322	acc: 0.39793
iteration: 110	loss: 1.65128	perp: 5.221	acc: 0.42971
iteration: 120	loss: 1.46384	perp: 4.329	acc: 0.48664
iteration: 130	loss: 1.27633	perp: 3.589	acc: 0.55245
iteration: 140	loss: 1.08611	perp: 2.967	acc: 0.63328
iteration: 150	loss: 0.89477	perp: 2.450	acc: 0.71469
iteration: 160	loss: 0.70791	perp: 2.033	acc: 0.78290
iteration: 170	loss: 0.53139	perp: 1.703	acc: 0.85610
iteration

In [29]:
torch.save(model.state_dict(), "../results/models/face")

In [35]:
idx_in_batch = 0
model.load_state_dict(torch.load("../results/models/face"))
model.eval()
pred = model.predict(
    {"vertices": [inputs["vertices"][idx_in_batch]]},
    max_seq_len=101,
)
pred

0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 

[tensor([203, 202, 200, 201]),
 tensor([203, 201, 147, 143,  97, 101,   1,   3]),
 tensor([203, 195, 194, 202]),
 tensor([203,   3,   5, 195]),
 tensor([202, 194,   4,   2]),
 tensor([202,   2,   0,  98,  94, 140, 144, 200]),
 tensor([201, 200, 144, 145, 184, 185, 146, 147]),
 tensor([199, 198, 196, 197]),
 tensor([199, 197,   7,   9]),
 tensor([199, 193, 192, 198]),
 tensor([199,   9,  19, 193]),
 tensor([198, 192,  18,   8]),
 tensor([198,   8,   6, 196]),
 tensor([197, 196, 194, 195]),
 tensor([197, 195,   5,   7]),
 tensor([196,   6,   4, 194]),
 tensor([193, 183, 182, 192]),
 tensor([193,  19,  25, 183])]

In [36]:
true = torch.cat(inputs["faces"][idx_in_batch])
true

tensor([203, 202, 200, 201, 203, 201, 147, 143,  97, 101,   1,   3, 203, 195,
        194, 202, 203,   3,   5, 195, 202, 194,   4,   2, 202,   2,   0,  98,
         94, 140, 144, 200, 201, 200, 144, 145, 184, 185, 146, 147, 199, 198,
        196, 197, 199, 197,   7,   9, 199, 193, 192, 198, 199,   9,  19, 193,
        198, 192,  18,   8, 198,   8,   6, 196, 197, 196, 194, 195, 197, 195,
          5,   7, 196,   6,   4, 194, 193, 183, 182, 192, 193,  19,  25, 183,
        192, 182,  24,  18, 191, 190, 178, 179, 191, 189, 188, 190, 191, 179,
         21,  17, 191,  17,  15, 189, 190, 188,  14,  16, 190,  16,  20, 178,
        189, 187, 186, 188, 189,  15,  13, 187, 188, 186,  12,  14, 187, 185,
        184, 186, 187,  13,  11, 100,  96, 142, 146, 185, 186, 184, 145, 141,
         95,  99,  10,  12, 183, 177, 176, 182, 183,  25,  29, 177, 182, 176,
         28,  24, 181, 180, 174, 175, 181, 179, 178, 180, 181, 175,  27,  23,
        181,  23,  21, 179, 180, 178,  20,  22, 180,  22,  26, 1

In [45]:
torch.cat(pred)

tensor([203, 202, 200, 201, 203, 201, 147, 143,  97, 101,   1,   3, 203, 195,
        194, 202, 203,   3,   5, 195, 202, 194,   4,   2, 202,   2,   0,  98,
         94, 140, 144, 200, 201, 200, 144, 145, 184, 185, 146, 147, 199, 198,
        196, 197, 199, 197,   7,   9, 199, 193, 192, 198, 199,   9,  19, 193,
        198, 192,  18,   8, 198,   8,   6, 196, 197, 196, 194, 195, 197, 195,
          5,   7, 196,   6,   4, 194, 193, 183, 182, 192, 193,  19,  25, 183])

In [44]:
min_len = min(torch.cat(pred).shape[0], true.shape[0])
acc = (torch.cat(pred)[:min_len] == true[:min_len]).sum() / min_len
acc

tensor(1.)

In [46]:
idx_in_batch = 1
model.load_state_dict(torch.load("../results/models/face"))
model.eval()
pred = model.predict(
    {"vertices": [inputs["vertices"][idx_in_batch]]},
)
pred

0, 1, 2, 3, 4, 5, 6, 7, 8, 

[tensor([61, 60, 58, 59]), tensor([61, 59,  1])]

In [47]:
true = torch.cat(inputs["faces"][idx_in_batch])
true

tensor([61, 60, 58, 59, 61, 59,  1,  3, 55, 61, 55, 53,  4, 60, 60,  4,  2,  0,
        58, 59, 58,  0,  1, 48, 10,  9, 47, 59, 47, 48,  1, 57, 56, 53, 54, 57,
        54, 42, 30, 24, 26, 18, 16, 28,  6,  8, 57,  8, 52, 51, 13,  7, 56, 56,
         7,  5, 21, 33, 31, 43, 35, 37, 45, 53, 55, 54, 53, 55,  3,  2,  4, 53,
         5,  6, 54, 54,  6, 28, 40, 42, 53, 45, 43, 31, 19, 21,  5, 52, 50, 49,
        51, 52, 14, 12, 50, 52,  8,  7, 13, 14, 51, 49, 11, 13, 50, 48, 47, 49,
        50, 25, 23, 29, 41, 39, 27, 10, 48, 50, 12, 11, 49, 50, 12, 10, 27, 29,
        23, 15, 17, 25, 49, 47, 44, 46, 38, 34, 22, 11, 47,  9, 11, 22, 20, 32,
        34, 38, 36, 44, 46, 45, 37, 38, 46, 44, 43, 45, 44, 36, 35, 43, 42, 41,
        29, 30, 42, 40, 39, 41, 40, 28, 27, 39, 38, 37, 35, 36, 34, 33, 21, 22,
        34, 32, 31, 33, 32, 20, 19, 31, 30, 29, 27, 28, 30, 28, 16, 24, 26, 25,
        17, 18, 26, 24, 23, 25, 24, 16, 15, 23, 22, 21, 19, 20, 18, 17, 15, 16,
        14, 13, 11, 12, 12, 11,  9, 10, 

In [49]:
min_len = min(torch.cat(pred).shape[0], true.shape[0])
acc = (torch.cat(pred)[:min_len] == true[:min_len]).sum() / min_len
acc

tensor(1.)